# 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256,10))

In [3]:
X = torch.rand(2, 20)

In [4]:
net(X)

tensor([[ 0.1618, -0.0901, -0.1610,  0.1158,  0.1683,  0.0073, -0.1686, -0.0007,
         -0.0976, -0.2782],
        [ 0.2203, -0.0321, -0.3254, -0.0316,  0.3107, -0.0478, -0.1845,  0.0938,
          0.0011, -0.3159]], grad_fn=<AddmmBackward0>)

In [5]:
X_no_batch = torch.rand(20)
net(X_no_batch)

tensor([ 0.1814, -0.0706, -0.2040,  0.0268,  0.2581, -0.0777, -0.1023,  0.0570,
         0.0465, -0.2824], grad_fn=<ViewBackward0>)

# 自定义块

In [7]:
"""
nn.Moudle 继承了一些很好用的类

所有的moudle有两个比较好用的函数，
一个是__init__,在里面定义哪些类和参数

"""
class MLP(nn.Module):
    def __init__(self):
        super().__init__() # 调用父类的，就是nn.Moudle，把一些内部所需要的参数全部设置好，初始化weight啊全部弄好
        # 定义两个隐藏层
        self.hidden = nn.Linear(20, 256) # 存在类的成员变量里面
        self.out = nn.Linear(256, 10) # 存在类的成员变量里面，名字就随便怎么定义了

    def forward(self, X): # self 就是python编程指向自己的那个东西 输入是我们的X
        return self.out(F.relu(self.hidden(X))) # F 里面实现了大量的常用的一些函数 ，做了激活后 到out里

In [10]:
net = MLP()
net(X)

tensor([[-0.0698,  0.1312, -0.0775,  0.1454,  0.0396,  0.0869, -0.1455, -0.0200,
          0.0820,  0.0556],
        [-0.0028,  0.1275,  0.0091,  0.2196,  0.2538, -0.0036, -0.3570, -0.1665,
         -0.0899,  0.3254]], grad_fn=<AddmmBackward0>)

In [11]:
print(X.shape)
print(net(X).shape)

torch.Size([2, 20])
torch.Size([2, 10])


In [9]:
net(X_no_batch)

tensor([-2.7178e-04, -1.5767e-01,  3.8272e-01, -1.8283e-01, -2.0700e-02,
         7.6819e-02, -3.1900e-01,  5.1230e-02,  4.1515e-02,  4.8142e-02],
       grad_fn=<ViewBackward0>)

In [12]:
print(X_no_batch.shape)
print(net(X_no_batch).shape)

torch.Size([20])
torch.Size([10])


# 顺序块

In [ ]:
"""
自己实现 经常使用的 nn.sequential()
"""

In [17]:
class MySequential(nn.Module):
    def __init__(self, *args): # 这里不一样，多接受了一个 list of input arguments
        super().__init__()
        for block in args: # 这个好理解吧， 就是我们平常输入的那样，很多个层
            self._modules[block] = block # 然后放到一个特殊的成员变量里，一个特殊的容器，pytorch就会知道放在里面都是我需要的一些特殊的层
                                        # 就是一个Orderdictionary,按顺序的字典 ， 所以排列的顺序就和插入进去是一样的
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [19]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)
# 哈哈哈自己实现了一个seq... 太酷啦

tensor([[ 0.0565, -0.0323,  0.1675, -0.0619,  0.0326,  0.1844, -0.0448, -0.0229,
         -0.1054, -0.0008],
        [ 0.0573,  0.0516,  0.2720, -0.0485, -0.0695,  0.2480, -0.0728,  0.0652,
         -0.0382,  0.0541]], grad_fn=<AddmmBackward0>)

# 在正向传播中执行代码

In [21]:
""""好处是当sequential满足不了自己的需求的时候，可以方便的自定义"""

'"好处是当sequential满足不了自己的需求的时候，可以方便的自定义'

In [22]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

# 通过从继承nn.Module 可以灵活地

tensor(-0.2287, grad_fn=<SumBackward0>)

# 混合搭配各种组合快的方法

In [25]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X)) # 疯狂套娃

chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP()) # 疯狂疯狂套娃
chimera(X)

tensor(0.2992, grad_fn=<SumBackward0>)

In [ ]:
""""虽然比sequentiall麻烦一点，但是可以灵活组合各种块，（论文代码都是这样的，所以已经习惯了，反而是sequential见不到哈哈哈）"""